# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,invercargill,-46.4000,168.3500,35.02,91,91,11.01,NZ,1726506749
1,1,flying fish cove,-10.4217,105.6791,76.68,83,100,8.05,CX,1726506750
2,2,niafunke,15.9322,-3.9906,90.54,47,69,11.45,ML,1726506751
3,3,west island,-12.1568,96.8225,78.78,73,73,19.57,CC,1726506752
4,4,ayaviri,-14.8864,-70.5889,64.26,23,38,6.80,PE,1726506753


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_data_df["Humidity"] = city_data_df["Humidity"].astype(float)

map_plot_1 = city_data_df.hvplot.points(
    "Lat",
    "Lng",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
#     size = "Humidity",
#     scale = 0.01,
#     color = "City"
)

#  For some reason, whenever I include the size parametere in the map plot, it gives an error


# Display the map
map_plot_1


C:\Users\husey\anaconda3\envs\dev\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lat,Lng]

In [4]:
filtered_weather_df = city_data_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 90 degrees but higher than 80.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 90) & (filtered_weather_df["Max Temp"] > 80)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,albany,42.6001,-73.9662,80.82,55.0,0,1.01,US,1726506776
89,89,cabo san lucas,22.8909,-109.9124,85.75,65.0,0,9.22,MX,1726506848
127,127,golden valley,45.0097,-93.3491,86.14,61.0,0,9.22,US,1726506889
128,128,hamilton,39.1834,-84.5333,86.77,44.0,0,9.22,US,1726506737
159,159,griffith,41.5284,-87.4237,87.94,36.0,0,4.61,US,1726506928
179,179,roma,43.2128,-75.4557,82.69,58.0,0,1.01,US,1726506686
195,195,arar,30.9753,41.0381,89.62,20.0,0,6.91,SA,1726506966
196,196,alpena,45.0617,-83.4327,84.52,39.0,0,8.05,US,1726506967
276,276,dibba al-hisn,25.6196,56.2729,87.35,59.0,0,2.35,AE,1726507052
291,291,collinsville,38.6703,-89.9846,87.89,58.0,0,3.00,US,1726507068


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = filtered_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(loc=0, column='Hotel Name',value = {})

# Display sample data
hotel_df

,Hotel Name,City,Country,Lat,Lng,Humidity
25,NaN,albany,US,42.6001,-73.9662,55.0
89,NaN,cabo san lucas,MX,22.8909,-109.9124,65.0
127,NaN,golden valley,US,45.0097,-93.3491,61.0
128,NaN,hamilton,US,39.1834,-84.5333,44.0
159,NaN,griffith,US,41.5284,-87.4237,36.0
179,NaN,roma,US,43.2128,-75.4557,58.0
195,NaN,arar,SA,30.9753,41.0381,20.0
196,NaN,alpena,US,45.0617,-83.4327,39.0
276,NaN,dibba al-hisn,AE,25.6196,56.2729,59.0
291,NaN,collinsville,US,38.6703,-89.9846,58.0


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories" : categories,
    "apiKey" : geoapify_key
}
name_adress = []
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)
    

    
    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
albany - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
golden valley - nearest hotel: Holiday Inn Express & Suites
hamilton - nearest hotel: North Vista Manor
griffith - nearest hotel: Quality Inn & Suites Hammond
roma - nearest hotel: Hampton Inn
arar - nearest hotel: دار أوتيل Dar Otel
alpena - nearest hotel: Holiday Inn Express & Suites
dibba al-hisn - nearest hotel: No hotel found
collinsville - nearest hotel: Drury Inn & Suites Collinsville
sanya - nearest hotel: No hotel found
big spring - nearest hotel: Settles Hotel
jimenez - nearest hotel: No hotel found
umluj - nearest hotel: No hotel found
las veredas - nearest hotel: Hotel Aeropuerto
ingeniero guillermo n. juarez - nearest hotel: Hotel El Indiano
alvaraes - nearest hotel: Hotel Divino


,Hotel Name,City,Country,Lat,Lng,Humidity
25,No hotel found,albany,US,42.6001,-73.9662,55.0
89,Comfort Rooms,cabo san lucas,MX,22.8909,-109.9124,65.0
127,Holiday Inn Express & Suites,golden valley,US,45.0097,-93.3491,61.0
128,North Vista Manor,hamilton,US,39.1834,-84.5333,44.0
159,Quality Inn & Suites Hammond,griffith,US,41.5284,-87.4237,36.0
179,Hampton Inn,roma,US,43.2128,-75.4557,58.0
195,دار أوتيل Dar Otel,arar,SA,30.9753,41.0381,20.0
196,Holiday Inn Express & Suites,alpena,US,45.0617,-83.4327,39.0
276,No hotel found,dibba al-hisn,AE,25.6196,56.2729,59.0
291,Drury Inn & Suites Collinsville,collinsville,US,38.6703,-89.9846,58.0


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lat",
    "Lng",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
#     size = "Humidity",
#     scale = 0.01,
#     color = "City"
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lat,Lng]